<a href="https://colab.research.google.com/gist/sadov/f550f1053ca79b931038c33b1a40714c/dateno_llm_wf1-qery_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Первый этап workflow -- Query Assistant LLM-агент

LLM-агент Query Assistant -- анализирует исходный запрос пользователя, определяет ключевые сущности, тему и контекст поиска. Формулирует уточняющие вопросы для устранения неоднозначности и сбора дополнительной информации. Переформулирует запрос в набор запросов для LLM-агента Dateno Search.

In [ ]:
HF_TOKEN='YOUR DATENO HF SPACE TOKEN'

In [ ]:
!pip install git+https://github.com/datenoio/datenollm

  Cloning https://github.com/datenoio/datenollm to /tmp/pip-req-build-g4h5l31r
  Running command git clone --filter=blob:none --quiet https://github.com/datenoio/datenollm /tmp/pip-req-build-g4h5l31r
  Resolved https://github.com/datenoio/datenollm to commit be85bb7dc5af956df40e91c5806a1aeac8f67055
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.5/213.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 6.8 MB/s eta 0:00:00
  Created wheel for datenollm: filename=datenollm-0.0.1-py3-none-any.whl size=26847 sha256=b350bea0bad86d2fa676c86d825e6e83c2e4dff71399d36f3be38b68c538f2f6
  Stored in 

In [ ]:
import json
from datenollm.client import DatenoClient
from datenollm.jupiter_nb import ask_llm, QueryAssistantChatWidget, get_full_path, QueriesSelector

Стартуем клиент для доступа к HF Space, где работает нужный нам LLM-агент. М.б. придется позапускать несколько раз если будут ошибки -- HF Space засыпает если долго не использовался и запускается опять при обращении к нему. При таком раскладе -- заводиться может пинка с третьего.

In [ ]:
client = DatenoClient('datenoio/explainable-query-assistant', hf_token=HF_TOKEN)

Loaded as API: https://datenoio-explainable-query-assistant.hf.space ✔


Проверим есть ли у нас доступ Google Disk'у. Если его нет, то последует запрос на выдачу разрешений для доступа к Google Account'у.

Google запросит очень много разрешений. Однако, если выставить только разрешение на доступ к файлам и каталогам, то подключения к Google Disk не происходит.

По умолчанию для сохранения в файлах истории и контекста мы используем только каталог /content/drive/MyDrive/colab_data/dateno/, ну или каталог указанный в переменной среды окружения `DRIVE_PATH`, ничего больше. См. https://github.com/datenoio/datenollm/blob/main/datenollm/file_utils.py

Если условие предоставления доступа к данным на персональном Google Disk'е является критическим, можно порекомендовать запускать данный Google Collab notebook на другом аккаунте Google, где важных данных нет.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


После предоставления доступа проверяем есть ли каталог /content/drive/MyDrive/colab_data/dateno/, если нет -- он будет создан. Для чистоты эксперимента можно удалить файлы которые имеются в данном каталоге или перенести их в другой каталог.

In [ ]:
get_full_path('')

'/content/drive/MyDrive/colab_data/dateno/'

## Эксперименты с LLM-агентом

Проверяем гипотезу что специализированный Chain of Tought (CoT) из специального набора вопросов может быть эффективнее чем обычные запросы к LLM.

Первым делом надо сформулировать вопрос с которым будет работать LLM Query Assistant.

In [ ]:
# @title Задайте вопрос:
query = "the trade turnover between Armenia and Cyprus" # @param {"type":"string","placeholder":"Your query"}

if not query:
  raise ValueError('Query is empty')

main_query = query

## Обычные запросы к LLM

Тут CoT не используем, а сразу отсылаем наш запрос в LLM.

Историю будем писать в такой файл:

In [ ]:
history_file = "history-eqa-llm.json"

In [ ]:
# @title 1-ый запрос отсылаем сразу в LLM

if not main_query:
  raise ValueError('Query is empty')

query, llm_response, history, error = ask_llm(client, main_query, history_file=history_file)
rating_widget = QueryAssistantChatWidget(client, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history-eqa-llm.json'


In [ ]:
# @title Ну и дальше общаемся с LLM как обычно
query = "for last 2 years" # @param {"type":"string","placeholder":"Your query"}

if not query:
  raise ValueError('Query is empty')

query, llm_response, history, error = ask_llm(client, query, history_file=history_file)
rating_widget = QueryAssistantChatWidget(client, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history-eqa-llm.json'


## Запрос через Chain of Tought (CoT)

Историю для CoT пишем сюда:

In [ ]:
history_file = "history-eqa-cot.json"

In [ ]:
# @title Отправить 1-ый CoT запрос к LLM:

if not main_query:
  raise ValueError('Query is empty')

llm_query = f"""
I want to analyze the query:
{main_query}

As a professional economic analyst, tell me what methodological approaches are optimal for this?
"""

query, llm_response, history, error = ask_llm(client, llm_query, history_file=history_file)
rating_widget = QueryAssistantChatWidget(client, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history-eqa-cot.json'


In [ ]:
# @title Отправить 2-ой CoT запрос к LLM

if not main_query:
  raise ValueError('Query is empty')

llm_query = f"""
Divide these methods into those that require a high level of expertise and special knowledge, and those that are based directly on data analysis.
"""

query, llm_response, history, error = ask_llm(client, llm_query, history_file=history_file)
rating_widget = QueryAssistantChatWidget(client, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history-eqa-cot.json'


In [ ]:
# @title Отправить 3-ий CoT запрос к LLM

if not main_query:
  raise ValueError('Query is empty')

llm_query = f"""
For Data Analysis methods, list the required indicators and possible names of the corresponding data sets.
"""

query, llm_response, history, error = ask_llm(client, llm_query, history_file=history_file)
rating_widget = QueryAssistantChatWidget(client, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history-eqa-cot.json'


In [ ]:
# @title Ну и дальше общаемся с LLM как обычно:
query = "for last 2 years" # @param {"type":"string","placeholder":"Your query"}

if not query:
  raise ValueError('Query is empty')

query, llm_response, history, error = ask_llm(client, query, history_file=history_file)
rating_widget = QueryAssistantChatWidget(client, history_file)
display(rating_widget.display())

history_file='/content/drive/MyDrive/colab_data/dateno/history-eqa-cot.json'


## Выбор запросов для следующего этапа workflow

Тут можем выбрать набор запросов который будет передан на следующий этап workflow. Список для выбора будет формироваться из того набора запросов который вы сформировали последним -- CoT или обычный запрос к LLM.

In [ ]:
# @title Выберите запросы:
llm_response = json.loads(llm_response)
selector_widget = QueriesSelector(llm_response['queries'])
display(selector_widget)

QueriesSelector(children=(Button(description='Select All', style=ButtonStyle()), Button(description='Select No…

Сохраняем выбранное в файле для передачи на следующий этап workflow:

In [ ]:
selector_widget.save_selected_queries('workflow_step1.json')

Selected queries saved to workflow_step1.json.


## Перейти к следующему разделу: ["Второй этап workflow -- Dateno Search LLM-агент"](https://gist.github.com/sadov/8d8954ee0635c491e9fcae25e27e1620#file-dateno_llm_wf2-dateno_search-ipynb)